In [3]:
!pip install langchain
!pip install pinecone
!pip install langchain_pinecone
!pip install openai
!pip install tiktoken
!pip install unstructured
!pip install sentence-transformers
!pip install faiss-gpu
!pip install -qU langchain-ollama
!pip install -U langchain-community
!curl curl -fsSL https://ollama.com/install.sh | sh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.2/407.2 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf

In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
PINECONE_KEY = user_secrets.get_secret("pdf_pinecone_api")
# PINECONE_KEY = user_secrets.get_secret("PINECONE_API_KEY")

In [3]:
import subprocess
subprocess.Popen(["ollama", "serve"])
import time
time.sleep(3) # Wait for a few seconds for Ollama to load!

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIOWWbcXdCGLJNj7SfYC+QM5+GQPoWXi0b2/XG/AZGxbr



2024/10/17 00:24:57 routes.go:1158: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://*] OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_DEVICES: http_proxy: https_proxy: no_proxy:]"
time=2024-10-17T00:24:57.174Z level=INFO source=images.go:754 msg="total blobs: 

In [2]:
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from pinecone import Pinecone, ServerlessSpec
import os
import bs4
from langchain.embeddings import HuggingFaceEmbeddings, CacheBackedEmbeddings
from langchain.document_loaders import WebBaseLoader
from langchain_pinecone import PineconeVectorStore
from langchain_pinecone import PineconeEmbeddings

from getpass import getpass
from uuid import uuid4
from langchain_core.documents import Document

os.environ["PINECONE_API_KEY"] = PINECONE_KEY


#Initialize LLM
llm = ChatOllama(
    model="llama3.1",
    temperature=0.5,
)
# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_KEY)

# Create or connect to the index
index_name = 'iskku-pdf'
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-west-2'
        )
    )
index = pc.Index(index_name)
#Embeddings
embeddings = PineconeEmbeddings(model="multilingual-e5-large")
vector_store = PineconeVectorStore(index = index, embedding=embeddings)


# #Documents loader and splitter
# splitter = CharacterTextSplitter.from_tiktoken_encoder(
#     separator="\n",
#     chunk_size=100,
#     chunk_overlap=100,
# )

# loader = WebBaseLoader(web_path = "https://www.skku.edu/eng/International/StudySKKU/Introduction.do")
# documents = loader.load_and_split(text_splitter=splitter)

# #Store the documents in the Pinecone index
# uuids = [str(uuid4()) for _ in range(len(documents))]

# vector_store.add_documents(documents=documents, ids = uuids)

#Use Pinecone vector-store as retriever
retriever= vector_store.as_retriever()

ModuleNotFoundError: No module named 'langchain_ollama'

In [6]:
from langchain.chains import RetrievalQA

template = """<s>[INST] Given the context - {context} </s>[INST] [INST] Answer the following question - {question}[/INST]"""
pt = PromptTemplate(
            template=template, input_variables=["context", "question"]
        )

In [7]:
from langchain.memory import ConversationSummaryMemory

rag = RetrievalQA.from_chain_type(
            llm=llm,
            retriever=retriever,
            memory=ConversationSummaryMemory(llm = llm),
            chain_type_kwargs={"prompt": pt, "verbose": True},
        )

/tmp/ipykernel_30/3105909023.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationSummaryMemory(llm = llm),


In [8]:
!ollama pull llama3.1

[GIN] 2024/10/17 - 00:25:50 | 200 |      48.585µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕                ▏    0 B/4.7 GB                  

time=2024-10-17T00:25:52.109Z level=INFO source=download.go:175 msg="downloading 8eeb52dfb3bb in 16 291 MB part(s)"


pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕                ▏  21 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   1% ▕                ▏  63 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   3% ▕                ▏ 145 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   5% ▕                ▏ 246 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   6% ▕█               ▏ 300 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   8% ▕█               ▏ 378 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...  10% ▕█               ▏ 485 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...  12% ▕█               ▏ 538 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...  14% ▕██              ▏ 644 MB/4.7 GB  580 MB/s      6spulling manifest 
pulling 8eeb

time=2024-10-17T00:26:22.350Z level=INFO source=download.go:175 msg="downloading 948af2743fc7 in 1 1.5 KB part(s)"


pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7...   0% ▕                ▏    0 B/1.5 KB                  pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743

...

pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12

time=2024-10-17T00:26:26.689Z level=INFO source=download.go:175 msg="downloading 56bb8bd477a5 in 1 96 B part(s)"


pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         
pulling 56bb8bd477a5...   0% ▕                ▏    0 B/  96 B                  pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B              

time=2024-10-17T00:26:28.484Z level=INFO source=download.go:175 msg="downloading 1a4c3c319823 in 1 485 B part(s)"


pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 1a4c3c319823...   0% ▕                ▏    0 B/ 485 B                  pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕████████████████▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 1a4c3c319823... 100% ▕████████████████▏  485 B                         pulling manifest 
pulling 8eeb52dfb3bb... 100% ▕████████████████▏ 4.7 GB                         
pulling 948af2743fc7... 100% ▕████████████████▏ 1.5 KB              

In [10]:
rag.invoke("Is it expensive to attend SKKU university?")['result']



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] Given the context - Tuition is about 4,000 USD per semester, and it varies depending on a student’s major and level of study.
	
			SKKU’s Office of International Relations (OIR) is the primary office responsible for the exchange program. In order to provide a well-organized academic experience to exchange students from partner universities, the OIR:

With more than 600 years of history and a unique culture, SKKU provides not only high-quality courses delivered by eminent professors and lecturers, 
		but also a unique campus ambience that you cannot find at other universities in Korea. Both Korean students studying at foreign universities and international students are welcome to apply for the program.
		Visiting students are given the same status as exchange students except for that tuition waivers are not provided to visiting students.

Visiting students have the chance 

"Based on the provided information, yes, it is expensive to attend SKKU university. The tuition fee for a semester is around 4,000 USD, which may vary depending on the student's major and level of study."